<a href="https://colab.research.google.com/github/micah-shull/LLMs/blob/main/LLM_053_RAG_CahsFlow4Cast_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### import Libraries

In [ ]:
!pip install -q sentence-transformers faiss-cpu
!pip install -q transformers load_dotenv huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

# Save to local Colab (optional)
csv_path = "/content/cleaned_blog_chunks.csv"
df_clean_chunks = pd.read_csv(csv_path)
df_clean_chunks

,title,filename,chunk_id,text
0,About Micah Shull,about_micah_shull.txt,0,About Micah Shull | Data Scientist & Founder o...
1,About Micah Shull,about_micah_shull.txt,1,That’s why I created Cashflow 4Cast — a servic...
2,Consistency That Builds Confidence,consistency_that_builds_confidence.txt,0,It Wasn’t Just One Store — It Was Every Store ...
3,Consistency That Builds Confidence,consistency_that_builds_confidence.txt,1,"And in every case, it consistently cut forecas..."
4,Consistency That Builds Confidence,consistency_that_builds_confidence.txt,2,MAPE (Mean Absolute Percentage Error): Shows f...
...,...,...,...,...
135,🤖 How it Works,🤖_how_it_works.txt,0,Forecasting Face-Off:Traditional Tools vs Mach...
136,🤖 How it Works,🤖_how_it_works.txt,1,Includes real-world economic indicators to ant...
137,🤖 How it Works,🤖_how_it_works.txt,2,"sales) 10+ (sales, category, date, promotions,..."
138,🤖 How it Works,🤖_how_it_works.txt,3,MAPE (Mean Absolute Percentage Error): Shows f...


,title,filename,chunk_id,text
0,About Micah Shull,about_micah_shull.txt,0,About Micah Shull | Data Scientist & Founder o...
1,About Micah Shull,about_micah_shull.txt,1,That’s why I created Cashflow 4Cast — a servic...
2,Consistency That Builds Confidence,consistency_that_builds_confidence.txt,0,It Wasn’t Just One Store — It Was Every Store ...
3,Consistency That Builds Confidence,consistency_that_builds_confidence.txt,1,"And in every case, it consistently cut forecas..."
4,Consistency That Builds Confidence,consistency_that_builds_confidence.txt,2,MAPE (Mean Absolute Percentage Error): Shows f...
...,...,...,...,...
135,🤖 How it Works,🤖_how_it_works.txt,0,Forecasting Face-Off:Traditional Tools vs Mach...
136,🤖 How it Works,🤖_how_it_works.txt,1,Includes real-world economic indicators to ant...
137,🤖 How it Works,🤖_how_it_works.txt,2,"sales) 10+ (sales, category, date, promotions,..."
138,🤖 How it Works,🤖_how_it_works.txt,3,MAPE (Mean Absolute Percentage Error): Shows f...



### ✅ Purpose of `search_blog()`

This function allows you to **ask a natural language question**, and it will:
1. **Embed your question** into vector space
2. **Search the FAISS index** for the most semantically similar blog chunks
3. **Print out the top-k results**, including the chunk, title, and source file

It’s like doing a Google-style search across your blog — but powered by meaning, not keywords.

---

### 🧠 Line-by-Line Breakdown

```python
import textwrap
```
- Used to wrap long text lines so results display cleanly.

---

```python
def search_blog(question, k=3, wrap_width=100):
```
- Defines the function.
- `question`: what the user is asking.
- `k`: how many top results to show.
- `wrap_width`: how wide the printed chunks should be.

---

```python
query_vec = embedder.encode([question])[0].astype("float32")
```
- Embeds the question using the same model as the blog chunks.
- Converts it into a format that FAISS can use to compare.

---

```python
D, I = index.search(np.array([query_vec]), k)
```
- Searches the FAISS index.
- Returns:
  - `I`: the indices of the top-k most similar blog chunks.
  - `D`: their distances (not used here).

---

```python
print(f"\n📌 Question: {question}\n{'=' * 80}")
```
- Displays the user's question clearly with formatting.

---

```python
for rank, idx in enumerate(I[0]):
    result = df_clean_chunks.iloc[idx]
```
- Loops through each of the top-k results.
- Retrieves the full metadata and content for each result.

---

```python
print(f"\n🔹 Result {rank + 1}")
print(f"📘 Title   : {result['title']}")
print("📄 Chunk   :")
print(textwrap.fill(result['text'], width=wrap_width))
print(f"🔗 Source  : {result['filename']}")
print("-" * 80)
```
- Nicely formats and prints each result:
  - Rank
  - Title
  - Chunk content
  - Source filename

---

### 📌 What You’re Learning Here

> This is how to connect embeddings, a FAISS index, and your content to create a **real-time semantic search tool**.

It’s the **retrieval half of RAG** — and a building block for intelligent assistants that cite sources.

Yes — you’re absolutely on the right track with your understanding. Let’s break this down clearly for your documentation.

---

### ✅ Do Embeddings Need to Match Shape?

**Yes — embeddings must match shape.**

```python
query_vec = embedder.encode([question])[0].astype("float32")
```

When you embed the question, it **must be embedded using the *same model*** that was used to embed the blog chunks (your documents).

Otherwise:
- The **dimensionality (shape)** won’t match.
- The **semantic meaning** won’t align.
- FAISS will throw an error, or worse, give you meaningless results.

> 📌 **Rule**: The query and the documents must be embedded by the **same model**, or at least models with **identical dimensions and vector spaces.**

---

### ✅ How Can You Check the Embedding Shape?

```python
print(doc_embeddings.shape)   # e.g., (1000, 384)
print(query_vec.shape)        # e.g., (384,)
```

If the second number doesn't match, the FAISS search will break.

---

### ✅ Why `k=3`?

```python
def search_blog(question, k=3, wrap_width=100):
```

- `k` = number of **top matching chunks** you want to retrieve from the index.
- You can change this to `k=5`, `k=10`, etc. depending on how broad or deep you want your search to be.

> Think of `k` as the number of results shown in a Google search.

---

### ✅ What is `wrap_width=100`?

- This sets the **max character width per line** for printing output.
- Helps format the chunks so they don’t scroll horizontally or appear as one giant paragraph.

```python
print(textwrap.fill(result['text'], width=wrap_width))
```

> 📌 If you're exporting to an app or UI, you can remove `textwrap`. But for notebooks or terminals, it improves readability.

---

### 🔁 Summary for Docs

> Embeddings must be created by the **same model** to ensure shape compatibility and semantic alignment. When using FAISS or similar vector search, the **query embedding must match** the shape and space of your indexed documents.
>
> - `k=3` retrieves the top 3 most relevant results.
> - `wrap_width=100` wraps the printed output so it’s easier to read in a notebook.



In [17]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings("ignore", message="The secret `HF_TOKEN` does not exist in your Colab secrets.*")


# Step 1: Load Sentence Transformer model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(model_name)

# Step 2: Embed all chunks
texts = df_clean_chunks["text"].tolist()
doc_embeddings = embedder.encode(texts, convert_to_numpy=True)

# Step 3: Create FAISS index and add vectors
embedding_dim = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(doc_embeddings.astype("float32"))

# Final confirmation
print(f"✅ Embedded {len(doc_embeddings)} chunks into {embedding_dim}-dimensional vectors.")
print("📦 FAISS index is ready for semantic search.")


✅ Embedded 140 chunks into 384-dimensional vectors.
📦 FAISS index is ready for semantic search.


### 💾 Save the FAISS Index and Metadata

In [14]:
# Create folders if needed
import os
os.makedirs("faiss_index", exist_ok=True)

# Save the FAISS index to disk
faiss.write_index(index, "faiss_index/blog_index.faiss")

# Save the DataFrame with metadata (title, filename, text, etc.)
df_clean_chunks.to_csv("faiss_index/blog_metadata.csv", index=False)

print("✅ FAISS index and metadata saved to 'faiss_index/' folder.")


✅ FAISS index and metadata saved to 'faiss_index/' folder.


### 🔁 Reload Later When Needed

In [ ]:
import faiss
import pandas as pd

# Reload FAISS index and metadata
index = faiss.read_index("faiss_index/blog_index.faiss")
df_clean_chunks = pd.read_csv("faiss_index/blog_metadata.csv")

print("✅ FAISS index and metadata reloaded.")


# 🧠 Semantic Search (QA via FAISS)

---

### ✅ How Does the Function Choose the Top‑k Results?

This line is the key:

```python
D, I = index.search(np.array([query_vec]), k)
```

Here’s what’s happening:

- `index.search(...)` takes the embedded question (`query_vec`) and:
  1. **Computes the distance** between your query and every document vector in the FAISS index.
  2. **Ranks them by similarity** — closest vectors (i.e., most semantically similar) come first.
  3. **Returns the indices (`I`) and distances (`D`)** of the top-k nearest neighbors.

---

### 📐 What Kind of Distance Is Used?

In this example:

```python
index = faiss.IndexFlatL2(embedding_dim)
```

You're using `IndexFlatL2`, which means:
- The similarity is computed using **L2 distance** (Euclidean distance).
- Smaller distance = closer = more relevant.

You could also use **cosine similarity**, but `L2` is commonly used and fast for many embedding models.

---

### 🔢 What Do `D` and `I` Mean?

- `D` = distances of the top-k results → lower is better.
- `I` = indices of those matching chunks in your `df_clean_chunks` DataFrame.

---

### ✅ Why Does This Work?

Because the question and the document chunks are all represented as **vectors in the same semantic space**:
- Similar ideas live close together.
- The FAISS search finds which blog chunks are **nearest to the meaning of your question**, not just word matches.

---

### 🧠 Summary for Docs

> The function uses FAISS to perform **semantic nearest neighbor search**. It finds the top‑k blog chunks whose vector embeddings are **closest** to the question’s embedding in vector space (using L2 distance). This allows it to return results that are **most semantically relevant**, even if they don’t share exact words.



In [15]:
import textwrap

def search_blog(question, k=3, wrap_width=100):
    # Embed the user query
    query_vec = embedder.encode([question])[0].astype("float32")

    # Search top-k results
    D, I = index.search(np.array([query_vec]), k)

    print(f"\n📌 Question: {question}\n{'=' * 80}")

    for rank, idx in enumerate(I[0]):
        result = df_clean_chunks.iloc[idx]
        print(f"\n🔹 Result {rank + 1}")
        print(f"📘 Title   : {result['title']}")
        print("📄 Chunk   :")
        print(textwrap.fill(result['text'], width=wrap_width))
        print(f"🔗 Source  : {result['filename']}")
        print("-" * 80)

In [16]:
search_blog("What economic indicators should I monitor as a small business?")


📌 Question: What economic indicators should I monitor as a small business?

🔹 Result 1
📘 Title   : 🚀 Looking Ahead: The Power of Economic Indicators in Forecasting
📄 Chunk   :
Why Economic Indicators Matter Looking Ahead: The Power of Economic Indicators in Forecasting Most
businesses use Excel or QuickBooks forecasts that only look inward — past sales, simple averages,
and outdated assumptions. We look outward. “You’re modeling what's coming, not just what already
happened.” That simple difference makes all the difference when it comes to preparing for risk,
spotting opportunity, and staying one step ahead. 📊 What We Include That Excel Doesn’t Local
Employment Trends (Retail, Manufacturing, Hospitality) Alachua County Median Household Income
Average Weekly Wages in Gainesville Per Capita Income in Alachua Federal Reserve Interest Rates
Inflation & Loan Delinquency Rates Statewide Unemployment & Payrolls These factors shape the
financial environment your customers live in — and your s

That response looks *too focused on one post*, when your knowledge base spans dozens of entries. We want it to feel like a **real assistant**, not just a blog highlighter.

---

## ✅ Goal: Make Answers Feel More Holistic

Right now, we're retrieving and printing **individual chunks**, but you want something that feels more:

- 🧠 **Integrated** — combines info across multiple posts
- 💬 **Conversational** — gives a real answer, not a quote
- 🧾 **Cited** — points to the original blog post(s) as backup

---

Fantastic question — this is where we move from prototyping in Colab to **deploying your RAG-powered chatbot** for real users.

---

### ✅ Your Goal:  
Run a chatbot that uses your **FAISS + blog chunks** to answer questions 24/7, likely via your website or app.

---

### 🧱 Key Components of a 24/7 Chatbot

| Component         | Purpose |
|------------------|---------|
| 🔍 **Vector Index (FAISS)** | Fast document retrieval |
| 📚 **Blog Chunks**          | Your knowledge base |
| 🤖 **LLM** (like Falcon or OpenAI) | Generates the response |
| 💬 **Chat UI / API**        | User-facing interface |
| ⚙️ **Backend App** (e.g., FastAPI) | Runs RAG logic and serves responses |
| ☁️ **Hosting Platform**     | Keeps your app running 24/7 |

---

### ✅ Deployment Blueprint

#### 1. **Convert Your Colab Workflow Into a Python Script**

Your notebook logic (embedding, loading FAISS, RAG function, etc.) should be refactored into:
- `rag_pipeline.py` — all logic
- `app.py` — web app interface (e.g., FastAPI or Flask)

---

#### 2. **Set Up a Simple API Server (e.g., FastAPI)**

```python
# app.py
from fastapi import FastAPI, Request
from rag_pipeline import search_and_generate

app = FastAPI()

@app.post("/ask")
async def ask(request: Request):
    data = await request.json()
    question = data.get("question")
    answer = search_and_generate(question)
    return {"question": question, "answer": answer}
```

---

#### 3. **Host It Somewhere (Always On)**

| Option         | Cost       | Notes |
|----------------|------------|-------|
| 🔄 **Replit**  | Free       | Easy to set up, always-on with ping |
| 🌍 **Render**  | Free tier  | Deploy FastAPI/Flask easily |
| ☁️ **Hugging Face Spaces** | Free (CPU) | Host a Gradio UI directly |
| 🔥 **Modal/Replicate** | Pay-per-use | Scale up dynamically |
| 🚀 **Docker + VPS (Advanced)** | ~$5/mo | Host it yourself |

---

#### 4. **Optional: Add a UI (Gradio or Chat Widget)**

To make it interactive:
- Use **Gradio** (easiest)
- Or integrate with your **blog site** using a chat widget and fetch responses via API

---

### 🔁 Bonus: Refresh the Blog Automatically?

Later, you can:
- Set up a **daily or weekly cron job** to re-embed new posts
- Replace the FAISS index automatically

---

### 🧠 Summary for Docs

> To run a 24/7 chatbot using RAG on your blog, you’ll convert your notebook into a script, deploy it via an API (e.g., FastAPI), host it on a platform like Replit or Hugging Face Spaces, and optionally add a front-end chat UI. This allows users to ask questions anytime and get responses powered by your blog posts.




#### Clean up Memory

In [ ]:
import torch
torch.cuda.empty_cache()

#### Remove Widgets from Notebook to save to Github

In [ ]:
import json
from google.colab import drive
drive.mount('/content/drive')

# Path to your current notebook file (adjust if different)
notebook_path = "/content/drive/My Drive/LLM/LLM_053_RAG_CahsFlow4Cast_Embeddings.ipynb"


# Load the notebook JSON
with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = json.load(f)

# Remove the widget metadata if it exists
if 'widgets' in nb.get('metadata', {}):
    del nb['metadata']['widgets']

# Save the cleaned notebook
with open(notebook_path, 'w', encoding='utf-8') as f:
    json.dump(nb, f, indent=2)

print("Notebook metadata cleaned. Try saving to GitHub again.")
